# Project by Lennart Wallentin, lennartwallentin@gmail.com

Feature selection is the technique of finding the minimal subset of features that allows for the maximal predictive power. Feature selection has become a crucial preprocessing stage to overcome issues such as “the curse of dimensionality” and overfitting. Moreover, the feature selection techniques are particularly useful to reduce the data size because many classifiers scale in complexity with the dimensionality of the data. Feature selection also eases the understanding of the dataset’s structure by eliminating irrelevant and redundant features. Therefore, feature selection is widely used in nearly all data science pipelines. 

A benefit of using ensembles of decision tree methods like extreme gradient boosting (XGBoost) is that they provide estimates of feature importance from a trained model. Hence I have created functions that do a form of backward stepwise selection based on the feature importance and a set of other input values with the goal to return the number of features to keep (with the highest feature importance score) in regard to the AUC-score. 
<br>I have created four functions, two for a more interpretable way of doing feature selection and the other two functions are automated and suitable to use in a data science pipeline especially in those occasions when you want to update your model.</br> 

The steps for doing that is roughly as follows:

1 - Make sure to have done the data preparation, created train and test sets and completed the first XGBoost classifier (including evaluated the classifiers feature importance scores). It is that classifiers feature importance scores that are going to be used for the feature selection, and the classifiers AUC-score will be used as a benchmark. In this notebook I will only show step 2 and step 3, I just assume step 1 is completed. If you want a thoroughly look at how I done this step 1 check out my other projects e.g. https://github.com/lennartwallentin/passenger_satisfaction_stacking_anova/blob/main/passenger_satisfaction_stacking_anova_lennart_wallentin.ipynb 
  

2 - For both the first interpretable (<i>interpretable_feature_selection</i>) and automated function (<i>automated_feature_selection</i>) I conduct a search on a broad range of features starting by keeping 75% of the features with the highest feature importance score down to and including keeping 25% of the features. I choose this range because with skewed distributions (as feature importance scores from XGBoost often are) it’s smart to use the 75th and 25th percentile as a range. Since I use this broad range I remove 5 features at each iteration.
<br>If the AUC-score is higher than the set AUC-limit including its confidence interval: Using the interpretable function I store the AUC-score and the number of features in a list. For the automated function I store the lowest number of features in a parameter. If the AUC-limit including its confidence interval isn't reached at least once a warning message is returned and for the automated function the number equal to keeping all functions is returned by the function.</br> 


3 - With the two other interpretable (<i>interpretable_precise_feature_selection</i>) and automated functions (<i>automated_precise_feature_selection</i>) I conduct a more precise/narrow search for the optimal number of features to keep. I remove one feature per iteration and use a span initially set to + 5 down to - 5 features from the number you choose/get from the function (interpretable_feature_selection) or (automated_feature_selection). The procedure is otherwise pretty much the same as before, it returns a list with AUC-score and number of features for the interpretable function and a parameter with the lowest number of features for the automated function. I also return a warning message if no number of features in the span for some reason doesn't reach the auc_limit including its confidence interval. In that case the number you set as the starting number of features is returned by the automated function.               

NOTE: For these functions I use AUC-score as the performance metric to compare the classifier and the number of features to keep, but that could easily be replaced by a F1-score, True Negative Rate, Accuracy, Precision, Recall etc. 


This is the structure of this project:

    1. Interpretable Feature Selection 
    2. Automated Feature Selection

In [1]:
# import necessary packages 
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import xgboost as xgb
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE) 

#### Glossary:
Dataset for the examples - https://github.com/lennartwallentin/passenger_satisfaction_stacking_anova/blob/main/data_passenger_satisfaction_stacking_anova_lennart_wallentin.csv

clf_xgb_1 = XGBoost classifier used to get the feature importance scores for the feature selection

xgb_roc_auc_1 = The AUC-score from the first classifier (clf_xgb_1) the score is used as a benchmark. 

xgb_roc_auc_1 = 0.9834

## 1. Interpretable Feature Selection 

In [55]:
def interpretable_feature_selection(clf, auc_limit, importance_type='gain', ci=0.99, steps=5, xtrain=X_train, xtest=X_test, ytrain=y_train, ytest=y_test):
    """
    Interpretable feature selection for XGBoost classifiers.

    Parameters
    ----------
    clf
        XGBoost classifier used to get the feature importance 

    auc_limit
        AUC-score to use as benchmark

    importance_type
        Initial value set to 'gain', for tree model, it’s either 'gain', 'weight', 'cover', 'total_gain' or 'total_cover'

    ci
        Initial value set to 0.99. Preferred confidence interval in decimal of percentage, for the approximately normal distributed AUC. 

    steps
        Initial value set to 5. Corresponds to the number of features to remove at each iteration

    xtrain, xtest, ytrain, ytest
        Corresponds to the sklearn.model_selection.train_test_split() algorithm. Initial values set according to that algorithm's naming convention.
    """
    # get the feature importance
    feature_importance = clf.get_booster().get_score(importance_type=str(importance_type))
    feature_importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns = ['score']).score.sort_values(ascending=False)
    # calculate the lower end of the Z score based on the preferred confidence interval 
    z_score = ci - ((1.-ci)/2)
    # empty list to store AUC-scores and number of features 
    auc_num_features = []
    # copies of xtrain and xtest 
    org_xtrain = xtrain
    org_xtest = xtest

    # For loop checking number of features to keep, 
    # range from keeping 75% of the features with the highest feature importance score, 
    # down to and including keeping 25% of the features. 
    for num_features in range(sum(feature_importance_df <= np.quantile(feature_importance_df, 0.75)), sum(feature_importance_df <= np.quantile(feature_importance_df, 0.25))-1, - abs(steps)):
        features_keep = feature_importance_df.iloc[:num_features]
        xtrain = xtrain.loc[:,xtrain.columns.isin(features_keep.index)]
        xtest = xtest.loc[:,xtest.columns.isin(features_keep.index)] 
        # set XGBoost parameters
        params = {'booster' : 'gbtree', 'objective' : 'binary:logistic', 'eval_metric': 'logloss', 'seed': RANDOM_STATE}
        # preform a cross-validation 
        cv = xgb.cv(dtrain=xgb.DMatrix(xtrain, ytrain), params=params, nfold=5, num_boost_round=500, early_stopping_rounds=10, seed=RANDOM_STATE, shuffle=False)
        # get the the best cv iteration 
        iter_best = len(cv)
        # create classifier used in feature selection
        clf_fs = xgb.XGBClassifier(n_estimators=iter_best, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss')
        clf_fs.fit(xtrain, ytrain)
        # AUC-score
        fpr, tpr, threshold = metrics.roc_curve(ytest, clf_fs.predict_proba(xtest)[:,1]) 
        auc = metrics.auc(fpr, tpr)

        # if the clf_fs AUC-score is higher than or equal to the auc_limit multiplied by the z_score
        # then append auc and num_features to auc_num_features
        if auc >= auc_limit*z_score:
            auc_num_features += [(round(auc,4) ,num_features)] 

        # restore xtrain and train for the next for loop iteration 
        xtrain = org_xtrain
        xtest = org_xtest
    
    # if auc_num_features is still empty after all loops are done
    # print a warning message explaining that
    if not auc_num_features:
        import warnings
        warnings.warn(f'Can’t keep between 75% to 25% of the features and still get an AUC-score of {auc_limit}')
    
    return auc_num_features

In [56]:
# the AUC score from the classifier containing all features
round(xgb_roc_auc_1,4)

0.9834

In [57]:
# do the first feature selection to get a broad range of AUC-scores and corresponding number of features.  
feature_selection = interpretable_feature_selection(clf=clf_xgb_1, auc_limit=xgb_roc_auc_1)
# feature_selection sorted by AUC-score in descending order
sorted(feature_selection, reverse=True)

[(0.984, 53),
 (0.9836, 48),
 (0.9831, 43),
 (0.9829, 38),
 (0.9817, 33),
 (0.9804, 28),
 (0.9778, 23),
 (0.9739, 18)]

In [58]:
# decided that 0.98 is the AUC limit, 
# create variables with the AUC-score and the number of features that yields an AUC score at or closest to above 0.98. 
fs_auc, fs_num_features = sorted(feature_selection, reverse=True)[-3] # 0.9804 and 28

In [59]:
def interpretable_precise_feature_selection(clf, start_num_features, auc_limit, importance_type='gain', ci=0.99, span = 5, xtrain=X_train, xtest=X_test, ytrain=y_train, ytest=y_test):
    """
    Interpretable precise feature selection for XGBoost classifiers.

    Parameters
    ----------
    clf
        XGBoost classifier used to get the feature importance 

    start_num_features
        Number of features to use as median for the span 

    auc_limit
        AUC-score to use as benchmark

    importance_type
        Initial value set to 'gain', for tree model, it’s either 'gain', 'weight', 'cover', 'total_gain' or 'total_cover'

    ci
        Initial value set to 0.99. Preferred confidence interval in decimal of percentage, for the approximately normal distributed AUC. 

    span
        Initial value set to 5. Corresponds to the span of steps (+ and -) to search from the start_num_features  

    xtrain, xtest, ytrain, ytest
        Corresponds to the sklearn.model_selection.train_test_split() algorithm. Initial values set according to that algorithm's naming convention.
    """    
    # get the feature importance
    feature_importance = clf.get_booster().get_score(importance_type=str(importance_type))
    feature_importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns = ['score']).score.sort_values(ascending=False)
    # calculate the lower end of the Z score based on the preferred confidence interval
    z_score = ci - ((1.-ci)/2)
    # empty list to store AUC-scores and number of features 
    precise_auc_num_features = [] 
    # copies of xtrain and xtest
    org_xtrain = xtrain
    org_xtest = xtest

    # For loop checking number of features to keep (based on feature importance score), 
    # range from declared start_num_features + span, 
    # down to and including start_num_features - span, the step is set to -1 feature per iteration  
    for num_features in range(start_num_features + span, (start_num_features - span) -1, -1): 
        features_keep = feature_importance_df.iloc[:num_features]
        xtrain = xtrain.loc[:,xtrain.columns.isin(features_keep.index)]
        xtest = xtest.loc[:,xtest.columns.isin(features_keep.index)] 
        # set XGBoost parameters
        params = {'booster' : 'gbtree', 'objective' : 'binary:logistic', 'eval_metric': 'logloss', 'seed': RANDOM_STATE}
        # preform a cross-validation 
        cv = xgb.cv(dtrain=xgb.DMatrix(xtrain, ytrain), params=params, nfold=5, num_boost_round=500, early_stopping_rounds=10, seed=RANDOM_STATE, shuffle=False)
        # get the the best cv iteration 
        iter_best = len(cv)
        # create classifier used in feature selection
        clf_fs = xgb.XGBClassifier(n_estimators=iter_best, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss')
        clf_fs.fit(xtrain, ytrain)
        # AUC-score
        fpr, tpr, threshold = metrics.roc_curve(ytest, clf_fs.predict_proba(xtest)[:,1]) 
        auc = metrics.auc(fpr, tpr)

        # if the clf_fs AUC-score is higher than or equal to the auc_limit multiplied by the z_score
        # then append auc and num_features to precise_auc_num_features        
        if auc >= auc_limit*z_score:
            precise_auc_num_features += [(round(auc,4) ,num_features)]

        # restore xtrain and train for the next for loop iteration        
        xtrain = org_xtrain
        xtest = org_xtest

    # if precise_auc_num_features is still empty after all loops are done
    # print a warning message explaining that
    if not precise_auc_num_features:
        import warnings
        warnings.warn('Can’t find as few features to keep at this range and still obtain a high enough auc_limit. Try to change either start_num_features, span, auc_limit and/or ci.')
    
    return precise_auc_num_features

In [60]:
# conduct a more precise feature selection around fs_num_features
# investigate how few features that can be kept and still yield a AUC-score on or above 0.98
# that's why auc_limit is set to 0.98 and ci is set to 1
precise_feature_selection = interpretable_precise_feature_selection(clf=clf_xgb_1, start_num_features=fs_num_features, auc_limit=0.98, ci=1)

# precise_feature_selection sorted by AUC-score in descending order
sorted(precise_feature_selection, reverse=True)

[(0.9817, 33),
 (0.9816, 32),
 (0.9815, 31),
 (0.9815, 30),
 (0.9807, 29),
 (0.9804, 28),
 (0.98, 27)]

In [61]:
# overwrite the old variables with the new AUC score and corresponding number of features
# this is the number of feature that will be kept in the next classifier and will yield a AUC-score of 0.98 
fs_auc, fs_num_features = sorted(precise_feature_selection, reverse=True)[-1] # 0.98 and 27
fs_num_features

27

## 2. Automated Feature Selection

In [62]:
def automated_feature_selection(clf, auc_limit, importance_type='gain', ci=0.99, steps=5, xtrain=X_train, xtest=X_test, ytrain=y_train, ytest=y_test):
    """
    Automated feature selection for XGBoost classifiers.

    Parameters
    ----------
    clf
        XGBoost classifier used to get the feature importance 

    auc_limit
        AUC-score to use as benchmark

    importance_type
        Initial value set to 'gain', for tree model, it’s either 'gain', 'weight', 'cover', 'total_gain' or 'total_cover'

    ci
        Initial value set to 0.99. Preferred confidence interval in decimal of percentage, for the approximately normal distributed AUC. 

    steps
        Initial value set to 5. Corresponds to the number of features to remove at each iteration

    xtrain, xtest, ytrain, ytest
        Corresponds to the sklearn.model_selection.train_test_split() algorithm. Initial values set according to that algorithm's naming convention.
    """
    # get the feature importance
    feature_importance = clf.get_booster().get_score(importance_type=str(importance_type))
    feature_importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns = ['score']).score.sort_values(ascending=False)
    # calculate the lower end of the Z score based on the preferred confidence interval 
    z_score = ci - ((1.-ci)/2)
    # initiate parameter to store the number of features
    low_num_features = None
    # copies of xtrain and xtest 
    org_xtrain = xtrain
    org_xtest = xtest

    # For loop checking number of features to keep, 
    # range from keeping 75% of the features with the highest feature importance score, 
    # down to and including keeping 25% of the features. 
    for num_features in range(sum(feature_importance_df <= np.quantile(feature_importance_df, 0.75)), sum(feature_importance_df <= np.quantile(feature_importance_df, 0.25))-1, - abs(steps)):
        features_keep = feature_importance_df.iloc[:num_features]
        xtrain = xtrain.loc[:,xtrain.columns.isin(features_keep.index)]
        xtest = xtest.loc[:,xtest.columns.isin(features_keep.index)] 
        # set XGBoost parameters
        params = {'booster' : 'gbtree', 'objective' : 'binary:logistic', 'eval_metric': 'logloss', 'seed': RANDOM_STATE}
        # preform a cross-validation 
        cv = xgb.cv(dtrain=xgb.DMatrix(xtrain, ytrain), params=params, nfold=5, num_boost_round=500, early_stopping_rounds=10, seed=RANDOM_STATE, shuffle=False)
        # get the the best cv iteration 
        iter_best = len(cv)
        # create classifier used in feature selection
        clf_fs = xgb.XGBClassifier(n_estimators=iter_best, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss')
        clf_fs.fit(xtrain, ytrain)
        # AUC-score
        fpr, tpr, threshold = metrics.roc_curve(ytest, clf_fs.predict_proba(xtest)[:,1]) 
        auc = metrics.auc(fpr, tpr)

        # if the clf_fs AUC-score is higher than or equal to the auc_limit multiplied by the z_score
        # then assign num_features to low_num_features
        if auc >= auc_limit*z_score:
            low_num_features = num_features 

        # restore xtrain and train for the next for loop iteration 
        xtrain = org_xtrain
        xtest = org_xtest
    
    # if low_num_features is still null after all loops are done keep all features 
    # print a warning message explaining that
    if not low_num_features:
        import warnings
        warnings.warn('Keeping all features, since the auc_limit (incl confidence interval) was not reached when trying to keep between 75% to 25% of the feature ')
        # assign the number equivalent to keeping all features 
        low_num_features = len(org_xtrain.columns)
    
    return low_num_features

In [63]:
# the AUC score from the classifier containing all features
round(xgb_roc_auc_1,4)

0.9834

In [64]:
# do the first automated feature selection on a broad range from keeping 75% to 25% of the features, 
# jumping 5 features per iteration and a auc_limit with 100% confidence interval  
fs_num_features = automated_feature_selection(clf=clf_xgb_1, auc_limit=xgb_roc_auc_1, ci=1)
fs_num_features

48

In [65]:
def automated_precise_feature_selection(clf, start_num_features, auc_limit, importance_type='gain', ci=0.99, span = 5, xtrain=X_train, xtest=X_test, ytrain=y_train, ytest=y_test):
    """
    Automated precise feature selection for XGBoost classifiers.

    Parameters
    ----------
    clf
        XGBoost classifier used to get the feature importance 

    start_num_features
        Number of features to use as median for the span 

    auc_limit
        AUC-score to use as benchmark

    importance_type
        Initial value set to 'gain', for tree model, it’s either 'gain', 'weight', 'cover', 'total_gain' or 'total_cover'

    ci
        Initial value set to 0.99. Preferred confidence interval in decimal of percentage, for the approximately normal distributed AUC. 

    span
        Initial value set to 5. Corresponds to the span of steps (+ and -) to search from the start_num_features  

    xtrain, xtest, ytrain, ytest
        Corresponds to the sklearn.model_selection.train_test_split() algorithm. Initial values set according to that algorithm's naming convention.
    """    
    # get the feature importance
    feature_importance = clf.get_booster().get_score(importance_type=str(importance_type))
    feature_importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns = ['score']).score.sort_values(ascending=False)
    # calculate the lower end of the Z score based on the preferred confidence interval
    z_score = ci - ((1.-ci)/2)
    # parameter to store the number of features
    lowest_num_features = None 
    # copies of xtrain and xtest
    org_xtrain = xtrain
    org_xtest = xtest

    # For loop checking number of features to keep (based on feature importance score), 
    # range from declared start_num_features + span, 
    # down to and including start_num_features - span, the step is set to -1 feature per iteration  
    for num_features in range(start_num_features + span, (start_num_features - span) -1, -1): 
        features_keep = feature_importance_df.iloc[:num_features]
        xtrain = xtrain.loc[:,xtrain.columns.isin(features_keep.index)]
        xtest = xtest.loc[:,xtest.columns.isin(features_keep.index)] 
        # set XGBoost parameters
        params = {'booster' : 'gbtree', 'objective' : 'binary:logistic', 'eval_metric': 'logloss', 'seed': RANDOM_STATE}
        # preform a cross-validation 
        cv = xgb.cv(dtrain=xgb.DMatrix(xtrain, ytrain), params=params, nfold=5, num_boost_round=500, early_stopping_rounds=10, seed=RANDOM_STATE, shuffle=False)
        # get the the best cv iteration 
        iter_best = len(cv)
        # create classifier used in feature selection
        clf_fs = xgb.XGBClassifier(n_estimators=iter_best, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss')
        clf_fs.fit(xtrain, ytrain)
        # AUC-score
        fpr, tpr, threshold = metrics.roc_curve(ytest, clf_fs.predict_proba(xtest)[:,1]) 
        auc = metrics.auc(fpr, tpr)

        # if the clf_fs AUC-score is higher than or equal to the auc_limit multiplied by the z_score
        # then assign num_features to lowest_num_features        
        if auc >= auc_limit*z_score:
            lowest_num_features = num_features

        # restore xtrain and train for the next for loop iteration        
        xtrain = org_xtrain
        xtest = org_xtest

    # if lowest_num_features is still null after all loops are done
    # print a warning message explaining that
    if not lowest_num_features:
        import warnings
        warnings.warn('Can’t find as few features to keep at this range and still obtain a high enough auc_limit. Try to change either auc_limit and/or ci.')
        # assign the number equivalent to start_num_features
        lowest_num_features = start_num_features
    
    return lowest_num_features

In [66]:
# conducts an more precise feature selection around feature_selection
# once again with a auc_limit with 100% confidence interval 
fs_num_features = automated_precise_feature_selection(clf=clf_xgb_1, start_num_features=fs_num_features, auc_limit=xgb_roc_auc_1, ci=1)
fs_num_features # 47 i.e. keep 63.5% of the features and get a AUC-score of 0.9837

47